In [238]:
# Import the Earth Engine Python Package
import ee
import sys
import json
from landdegradation import stats
from landdegradation import util

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

In [215]:
#######################################################################################
# USER INPUT for testing, REMOVE LATER
year_start = 2001
year_start_str = '2001'
year_end   = 2015
year_end_str = '2015'
sa_name    = 'KEN'      # 'globe' or ISO3 code
method     = 'ndvi_trend' # 'ndvi_trend','prestrend', 'rue_trend', wue_trend'
sensor     = 'modis'      # 'modis' or 'avhrr'
climate    = 'prec_gpcp'  # 'prec_gpcp', 'prec_persian', 'prec_chirps', 'prec_gpcc', 'soilm_merra2', 'soilm_erai', 'et_mod16a2' 

In [173]:
# INPUT DATA
# Country boundaries
if (sa_name == 'globe'):
    sa = ee.Geometry.Polygon([-180, 80, 0, 80, 180, 80, 180, -80, 0, -80, -180, -80])
elif (sa_name != 'globe'):
    sa = ee.FeatureCollection("USDOS/LSIB/2013").filter(ee.Filter.eq('iso_alpha3', sa_name))

In [227]:
# Sensors        
if (sensor == 'avhrr'):
    ndvi_1yr = ee.Image("users/geflanddegradation/toolbox_datasets/ndvi_avhrr_1982_2015")
    pixel = 8000
elif (sensor == 'modis'):
    ndvi_1yr = ee.Image("users/geflanddegradation/toolbox_datasets/ndvi_modis_2001_2016")
    pixel = 250
else:
    sys.exit()

In [217]:
# Climate
if (climate == 'et_mod16a2'):
    climate_1yr = ee.Image("users/geflanddegradation/toolbox_datasets/et_modis_2000_2014")
if (climate == 'prec_gpcp'):
    climate_1yr = ee.Image("users/geflanddegradation/toolbox_datasets/prec_gpcp23_1979_2016")
if (climate == 'prec_gpcc'):
    climate_1yr = ee.Image("users/geflanddegradation/toolbox_datasets/prec_gpcc_1901_2016")
if (climate == 'prec_chirps'):
    climate_1yr =  ee.Image("users/geflanddegradation/toolbox_datasets/prec_chirps_1981_2016")
if (climate == 'prec_persian'):
    climate_1yr = ee.Image("users/geflanddegradation/toolbox_datasets/prec_persian_1983_2015")
if (climate == 'soilm_merra2'):
    climate_1yr = ee.Image("users/geflanddegradation/toolbox_datasets/soilm_merra2_1980_2016")
if (climate == 'soilm_areai'):
    climate_1yr = ee.Image("users/geflanddegradation/toolbox_datasets/soilm_erai_1979_2016")

In [218]:
# Land cover
landc = ee.Image("users/geflanddegradation/toolbox_datasets/lcov_esacc_1992_2015").select('y'+year_end_str)

In [219]:
# Define Kendall parameter values for a significance of 0.05
period = year_end - year_start + 1
coefficients = ee.Array([4, 6, 9, 11, 14, 16, 19, 21, 24, 26, 31, 33, 36,
                        40, 43, 47, 50, 54, 59, 63, 66, 70, 75, 79, 84,
                        88, 93, 97, 102, 106, 111, 115, 120, 126, 131,
                        137, 142])
kendall = coefficients.get([period - 4])

In [220]:
#Function to calc ndvi trend

if (method == 'ndvi_trend'):
    def f_img_coll(ndvi_stack):
        img_coll = ee.List([])
        for k in range(year_start, year_end):
            ndvi_img = ndvi_stack.select('y' + str(k)).addBands(ee.Image(k).float()).rename(['ndvi', 'year'])
            img_coll = img_coll.add(ndvi_img)
        return ee.ImageCollection(img_coll)
    
    ## Apply function to compute NDVI annual integrals from 15d observed NDVI data
    ndvi_1yr_coll = f_img_coll(ndvi_1yr)
    
    ## Compute linear trend function to predict ndvi based on year (ndvi trend)
    lf_trend = ndvi_1yr_coll.select(['year', 'ndvi']).reduce(ee.Reducer.linearFit())

    ## Compute Kendall statistics
    mk_trend  = stats.mann_kendall(ndvi_1yr_coll.select('ndvi'))

In [201]:
## Function to create image collection with annual NDVI integrals and years
"""
### translated direct
if (method == 'prestrend'):
    def f_img_coll(ndvi_stack):
        img_coll = ee.List([])
        for (k = year_start k <= year_end  k += 1):
        ndvi_img = ndvi_stack.select('y'+k)
                                .addBands(climate_1yr.select('y'+k))
                                .rename(['ndvi','clim']).set({'year': k})    
        img_coll = img_coll.add(ndvi_img)
    return ee.ImageCollection(img_coll)  
"""
if (method == 'prestrend'):
    def f_img_coll(ndvi_stack):
        img_coll = ee.List([])
        for k in range(year_start, year_end):
            ndvi = ndvi_1yr.filter(ee.Filter.eq('year', k)).select('ndvi').median()
            clim = climate_1yr.filter(ee.Filter.eq('year', k)).select('ndvi').median()
            img = ndvi.addBands(clim.addBands(ee.Image(k).float())).rename(['ndvi','clim','year']).set({'year': k})
            img_coll = img_coll.add(img)
            return ee.ImageCollection(img_coll)

    ## Function to predict NDVI from climate
    first = ee.List([])
    def f_ndvi_clim_p(image, list):
        ndvi = lf_clim_ndvi.select('offset').add((lf_clim_ndvi.select('scale').multiply(image))).set({'year': image.get('year')})
        return ee.List(list).add(ndvi)

    ## Function to compute residuals (ndvi obs - ndvi pred)
    def f_ndvi_clim_r_img(year): 
        ndvi_o = ndvi_1yr_coll.filter(ee.Filter.eq('year', year)).select('ndvi').median()
        ndvi_p = ndvi_1yr_p.filter(ee.Filter.eq('year', year)).median()
        ndvi_r = ee.Image(year).float().addBands(ndvi_o.subtract(ndvi_p))
        return ndvi_r.rename(['year','ndvi_res'])

    # Function to compute differences between observed and predicted NDVI and compilation in an image collection
    def stack(year_start, year_end):
        img_coll = ee.List([])
        for k in range(year_start, year_end):
            ndvi = ndvi_1yr_o.filter(ee.Filter.eq('year', k)).select('ndvi').median()
            clim = clim_1yr_o.filter(ee.Filter.eq('year', k)).select('ndvi').median()
            img = ndvi.addBands(clim.addBands(ee.Image(k).float())).rename(['ndvi','clim','year']).set({'year': k})
            img_coll = img_coll.add(img)
        return ee.ImageCollection(img_coll)

    ## Function create image collection of residuals
    def f_ndvi_clim_r_coll(year_start, year_end): 
        res_list = ee.List([])
        #for(i = year_start i <= year_end i += 1):
        for i in range(year_start, year_end):
            res_image = f_ndvi_clim_r_img(i)
            res_list = res_list.add(res_image)
        return ee.ImageCollection(res_list)

    ## Apply function to create image collection of ndvi and climate
    ndvi_1yr_coll = f_img_coll(ndvi_1yr)
    
    ## Compute linear trend function to predict ndvi based on climate (independent are followed by dependent var
    lf_clim_ndvi = ndvi_1yr_coll.select(['clim', 'ndvi']).reduce(ee.Reducer.linearFit())

    ## Apply function to  predict NDVI based on climate
    ndvi_1yr_p = ee.ImageCollection(ee.List(ndvi_1yr_coll.select('clim').iterate(f_ndvi_clim_p, first)))

    ## Apply function to compute NDVI annual residuals
    ndvi_1yr_r  = f_ndvi_clim_r_coll(year_start,year_end)

    print(ndvi_1yr_r)

    ## Fit a linear regression to the NDVI residuals
    lf_trend = ndvi_1yr_r.select(['year', 'ndvi_res']).reduce(ee.Reducer.linearFit())

    ## Compute Kendall statistics
    mk_trend  = stats.mann_kendall(ndvi_1yr_r.select('ndvi_res'))

In [202]:
## Function to create image collection with annual NDVI integrals and years
"""
if (method == 'rue_trend' or method == 'wue_trend' ):
    def f_img_coll(ndvi_stack):
        img_coll = ee.List([])
        for (k = year_start k <= year_end  k += 1):
            ndvi_img = ndvi_stack.select('y'+k).divide(climate_1yr.select('y'+k))
                                .addBands(ee.Image(k).float())
                                .rename(['rue','year']).set({'year': k})
        img_coll = img_coll.add(ndvi_img)
    return ee.ImageCollection(img_coll)
"""
if (method == 'rue_trend' or method == 'wue_trend' ):
    def f_img_coll(ndvi_stack):
        for k in range(year_start, year_end):
            ndvi = ndvi_1yr.filter(ee.Filter.eq('year', k)).select('ndvi').median()
            clim = climate_1yr.filter(ee.Filter.eq('year', k)).select('ndvi').median()
            img = ndvi.addBands(clim.addBands(ee.Image(k).float())).rename(['ndvi','clim','year']).set({'year': k})
            img_coll = img_coll.add(img)
   
    ## Apply function to compute rue and store as a collection
    rue_1yr_coll = f_img_coll(ndvi_1yr)

    ## Compute linear trend function to predict ndvi based on year (ndvi trend)
    lf_trend = rue_1yr_coll.select(['year', 'rue']).reduce(ee.Reducer.linearFit())

    ## Compute Kendall statistics
    mk_trend  = stats.mann_kendall(rue_1yr_coll.select('rue'))

In [210]:
## Create 2 band output raster
slope = lf_trend.select('scale')
attri = ee.Image(0).where(lf_trend.select('scale').gt(0)and(mk_trend.abs().gte(kendall)),  1)\
.where(lf_trend.select('scale').lt(0)and(mk_trend.abs().gte(kendall)), -1)\
.where(mk_trend.abs().lte(kendall), 0)\
.where(landc.eq(210),2)\
.where(landc.eq(190),3)
                       
output = slope.addBands(attri).rename(['slope','attri'])
#print(output)

In [247]:
export = {
    "image": output,
    "description": "ken_modis_ndvitrend",
    "bucket": "ldmt",
    "region": util.get_coords(json.loads(util.sen_geojson)),
    "maxPixels": 100000000000,
    "scale": pixel}

task = ee.batch.Export.image.toCloudStorage(**export)

In [248]:
task.start()

In [234]:
"""
export = {
        'image': lf_trend.select('scale').where(mk_trend.abs().lte(kendall), -99999).where(lf_trend.select('scale').abs().lte(0.000001), -99999).unmask(-99999),
        'description': EXECUTION_ID,
        'fileNamePrefix': EXECUTION_ID,
        'bucket': BUCKET,
        'maxPixels': 10000000000,
        'scale': 250,
        'region': util.sen_geojson
    }

logger.debug("Setting up task.")
task = ee.batch.Export.image.toCloudStorage(**export)

logger.debug("Starting task.")
task.start()
task_state = task.status().get('state')
while task_state == 'READY' or task_state == 'RUNNING':
    task_progress = task.status().get('progress', 0.0)
    # update GEF-EXECUTION progress
    logger.send_progress(task_progress)
    logger.debug("Task progress {}.".format(task_progress))
    # update variable to check the condition
    task_state = task.status().get('state')
    sleep(5)

logger.debug("Leaving integral_trends function.")
return "https://{}.storage.googleapis.com/{}.tif".format(BUCKET, EXECUTION_ID)"""

NameError: name 'EXECUTION_ID' is not defined

In [135]:
"""
def run(params, logger):
    """."""
    logger.debug("Loading parameters.")

    year_start = params.get('year_start', 2003)
    year_end = params.get('year_end', 2015)
    geojson = json.loads(params.get('geojson', util.sen_geojson))
    resolution = params.get('resolution', 250)
    dataset = params.get('dataset', 'AVHRR')

    # Check the ENV. Are we running this locally or in prod?
    if params.get('ENV') == 'dev':
        EXECUTION_ID = str(random.randint(1000000, 99999999))
    else:
        EXECUTION_ID = params.get('EXECUTION_ID', None)

    logger.debug("Running main script.")
    url = integral_trend(year_start, year_end, geojson, resolution, dataset, 
            EXECUTION_ID, logger)

    logger.debug("Setting up results JSON.")
    results_url = CloudUrl(url, 'TODO_HASH_GOES_HERE') 
    cloud_dataset = CloudDataset('geotiff', 'integral_trends', results_url)
    gee_results = GEEResults('cloud_dataset', cloud_dataset)
    results_schema = GEEResultsSchema()
    json_result = results_schema.dump(gee_results)

    logger.debug("Leaving run function.")
    return json_result.data
    """

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-135-f52eb156afb5>, line 30)